In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:

import zipfile

# Replace 'DATASET.zip' with the actual name of your zip file
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:     # replace it with ur zipfilename
    zip_ref.extractall('archive') # extracts to a directory called 'DATASET'

In [6]:
# Paths to dataset folders
TRAIN_DIR = '/content/archive/train'
TEST_DIR = '/content/archive/test'

# Image size and batch size
IMG_SIZE = (128, 128)  # Resize all images to 128x128
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'   # keep categorical for multiclassification
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  #keep categorical for multiclassification
)




Found 3200 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [7]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(8, activation='softmax')  # softmax for multi classification
    ])

    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',  #sparse_categorical_crossentropy for multiclassification
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()





#Training the Model


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,672 (12.61 MB)

 Trainable params: 3,305,672 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator
)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step - accuracy: 0.2316 - loss: 1.9502 - val_accuracy: 0.6475 - val_loss: 1.0819
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.6025 - loss: 1.0870 - val_accuracy: 0.7788 - val_loss: 0.6035
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.7490 - loss: 0.6817 - val_accuracy: 0.8213 - val_loss: 0.5135
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8122 - loss: 0.5170 - val_accuracy: 0.8838 - val_loss: 0.3735
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.8672 - loss: 0.3738 - val_accuracy: 0.8637 - val_loss: 0.3592


In [9]:
# Saving model
model.save('mango_model.h5')   #potato_disease.h5

In [10]:
# 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8657 - loss: 0.3569
Test Accuracy: 0.8637499809265137


In [ ]:
# below code is for multiclassification prediction do not execute it

In [11]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/mango_model.h5')  # Updated model path

# Path to your custom image
image_path = r"/content/archive/train/Powdery Mildew/20211109_121218 (Custom).jpg"  # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training

def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)

# Class labels (ensure they match the model's training)
class_labels = ["Anthracnose","Bacterial Canker","Cutting Weevil","Die Black","Gall Midge","Healthy","Powdery Mildew","Sooty Mould"]

# Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]

# Output the prediction
print(f"The image is predicted to be: {predicted_class_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
The image is predicted to be: Powdery Mildew
